In [1]:
!pip install torch torchvision torchsummary


Defaulting to user installation because normal site-packages is not writeable


In [3]:
# ---------------------------
# 1. Import Libraries
# ---------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
import time
import os







In [16]:
# ---------------------------
# 2. Enable GPU and Mixed Precision
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = torch.amp.GradScaler("cuda")  # Mixed precision scaler

# ---------------------------
# 3. Mount Google Drive (for model saving)
# ---------------------------


# Path to save models
#model_dir = "saved_model"
#os.makedirs(model_dir, exist_ok=True)
#model_path = os.path.join(model_dir, "efficientnetv2_s_disease.pth")
model_path=r"C:\Users\kumar\OneDrive\Desktop\SEM 6\GenAI\img\saved_model"


In [17]:
# ---------------------------
# 4. Data Loading and Preprocessing
# ---------------------------
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# ✅ Correct Dataset Path
data_dir = r"C:\Users\kumar\OneDrive\Desktop\SEM 6\GenAI\img\disease_dataset"
batch_size = 32
img_size = 224

# ✅ Transforms with Data Augmentation
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(img_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ✅ Since all classes are in the same root folder, use a single ImageFolder
# Split into train, val, and test sets
full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transforms)

# Split into 80% train, 10% val, 10% test
train_size = int(0.8 * len(full_dataset))
val_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    full_dataset, [train_size, val_size, test_size]
)

# Apply the correct transforms
train_dataset.dataset.transform = train_transforms
val_dataset.dataset.transform = val_test_transforms
test_dataset.dataset.transform = val_test_transforms

# ✅ Dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# ✅ Get class names
class_names = full_dataset.classes
print(f"\n✅ Classes: {class_names}")



✅ Classes: ['cotton_bacterial_blight', 'cotton_curl_virus', 'cotton_fussarium_wilt', 'cotton_healthy', 'maize_blight', 'maize_common_rust', 'maize_gray_leaf_spot', 'maize_healthy', 'rice_bacterial_leaf_blight', 'rice_blast', 'rice_brown_spot', 'rice_healthy', 'rice_tungro', 'wheat_brown_rust', 'wheat_fusarium_head_blight', 'wheat_healthy', 'wheat_mildew', 'wheat_septoria']


In [18]:

# ---------------------------
# 5. Model Definition (EfficientNetV2-S)
# ---------------------------
num_classes = len(class_names)

# Load EfficientNetV2-S with pretrained weights
model = models.efficientnet_v2_s(weights="IMAGENET1K_V1")
model.classifier[1] = nn.Linear(1280, num_classes)  # Adjust final layer for your classes
model = model.to(device)

# Loss, Optimizer, Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

# ---------------------------
# 6. Save Model Function
# ---------------------------
def save_model(model, model_path):
    """Save model to Google Drive"""
    torch.save(model.state_dict(), model_path)
    print(f"\n💾 Model saved at: {model_path}")

# ---------------------------
# 7. Evaluation Function
# ---------------------------
def evaluate_model(model, loader):
    """Evaluate model accuracy"""
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    acc = 100. * correct / total
    return acc

# ---------------------------
# 8. Training Function (with Mixed Precision)
# ---------------------------

def train_model(model, train_loader, val_loader, test_loader, num_epochs, start_epoch, model_folder):
    """Train the model with mixed precision and save with a unique filename at the end of each block."""

    print(f"\n🚀 Starting Training for {num_epochs} epochs...")

    for epoch in range(start_epoch, start_epoch + num_epochs):
        print(f"\n🔥 Epoch {epoch + 1}")

        model.train()
        running_loss, correct, total = 0.0, 0, 0

        start_time = time.time()

        for inputs, targets in tqdm(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            # Mixed precision training
            with torch.amp.autocast("cuda"):
                outputs = model(inputs)
                loss = criterion(outputs, targets)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            running_loss += loss.item()

        end_time = time.time()
        epoch_time = (end_time - start_time) / 60  # Time in minutes

        # ✅ Calculate accuracies
        train_acc = 100. * correct / total
        val_acc = evaluate_model(model, val_loader)
        test_acc = evaluate_model(model, test_loader)

        print(f"\n✅ Train Accuracy: {train_acc:.2f}% | Val Accuracy: {val_acc:.2f}% | Test Accuracy: {test_acc:.2f}% | Time: {epoch_time:.2f} min")

        # ✅ Scheduler step
        scheduler.step()

    # ✅ Save model with a unique name after the training block
    save_model(model, model_folder, start_epoch + num_epochs)


In [19]:
# ---------------------------
# ✅ Define Model Folder for Saving
model_folder = r"C:\Users\kumar\OneDrive\Desktop\SEM 6\GenAI\img\saved_model"

# ✅ Initial 12 Epochs
print("\n🚀 Starting Initial Training (12 epochs)...")

# Track the best validation accuracy
best_val_acc = 0.0

def train_model_with_best_save(model, train_loader, val_loader, test_loader, num_epochs, start_epoch, model_folder):
    """Train the model and save only the best model based on validation accuracy."""
    global best_val_acc  # Use the global variable to track the best validation accuracy

    print(f"\n🚀 Starting Training for {num_epochs} epochs...")

    for epoch in range(start_epoch, start_epoch + num_epochs):
        print(f"\n🔥 Epoch {epoch + 1}")

        model.train()
        running_loss, correct, total = 0.0, 0, 0

        start_time = time.time()

        for inputs, targets in tqdm(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            # Mixed precision training
            with torch.amp.autocast("cuda"):
                outputs = model(inputs)
                loss = criterion(outputs, targets)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            running_loss += loss.item()

        end_time = time.time()
        epoch_time = (end_time - start_time) / 60  # Time in minutes

        # ✅ Calculate accuracies
        train_acc = 100. * correct / total
        val_acc = evaluate_model(model, val_loader)
        test_acc = evaluate_model(model, test_loader)

        print(f"\n✅ Train Accuracy: {train_acc:.2f}% | Val Accuracy: {val_acc:.2f}% | Test Accuracy: {test_acc:.2f}% | Time: {epoch_time:.2f} min")

        # ✅ Save the model if validation accuracy improves
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            save_model(model, os.path.join(model_folder, "best_model.pth"))
            print(f"\n💾 New Best Model Saved with Val Accuracy: {best_val_acc:.2f}%")

        # ✅ Scheduler step
        scheduler.step()

# Start training
train_model_with_best_save(model, train_loader, val_loader, test_loader, num_epochs=6, start_epoch=0, model_folder=model_folder)

# User-controlled loop for additional training blocks
current_epoch = 12  # Keep track of the total epochs trained

while True:
    # Ask the user if they want to continue training
    user_input = input("\n❓ Do you want to continue training? (yes/no): ").strip().lower()

    if user_input == 'yes':
        try:
            extra_epochs = int(input("\n🔢 How many more epochs? (e.g., 3 or 5): ").strip())

            if extra_epochs <= 0:
                print("\n⚠️ Invalid input. Epochs must be > 0.")
                continue

            print(f"\n🔥 Continuing Training for {extra_epochs} More Epochs...")

            # Train for the additional epochs
            train_model_with_best_save(model, train_loader, val_loader, test_loader, num_epochs=extra_epochs,
                                       start_epoch=current_epoch, model_folder=model_folder)

            # Update the current epoch count
            current_epoch += extra_epochs

        except ValueError:
            print("\n⚠️ Invalid input. Please enter a valid number of epochs.")

    elif user_input == 'no':
        print("\n🚫 Training stopped by user.")
        break
    else:
        print("\n⚠️ Invalid input. Please type 'yes' or 'no'.")



🚀 Starting Initial Training (12 epochs)...

🚀 Starting Training for 6 epochs...

🔥 Epoch 1


100%|██████████| 241/241 [00:52<00:00,  4.57it/s]



✅ Train Accuracy: 73.31% | Val Accuracy: 79.94% | Test Accuracy: 79.54% | Time: 0.88 min

💾 Model saved at: C:\Users\kumar\OneDrive\Desktop\SEM 6\GenAI\img\saved_model\best_model.pth

💾 New Best Model Saved with Val Accuracy: 79.94%

🔥 Epoch 2


100%|██████████| 241/241 [00:52<00:00,  4.62it/s]



✅ Train Accuracy: 87.18% | Val Accuracy: 86.38% | Test Accuracy: 86.71% | Time: 0.87 min

💾 Model saved at: C:\Users\kumar\OneDrive\Desktop\SEM 6\GenAI\img\saved_model\best_model.pth

💾 New Best Model Saved with Val Accuracy: 86.38%

🔥 Epoch 3


...


✅ Train Accuracy: 99.09% | Val Accuracy: 95.32% | Test Accuracy: 95.02% | Time: 0.79 min

🔥 Epoch 30


100%|██████████| 241/241 [00:47<00:00,  5.05it/s]



✅ Train Accuracy: 99.22% | Val Accuracy: 93.76% | Test Accuracy: 93.87% | Time: 0.79 min

🔥 Epoch 31


100%|██████████| 241/241 [00:47<00:00,  5.10it/s]



✅ Train Accuracy: 99.69% | Val Accuracy: 95.63% | Test Accuracy: 96.57% | Time: 0.79 min

⚠️ Invalid input. Please type 'yes' or 'no'.

⚠️ Invalid input. Please type 'yes' or 'no'.

🚫 Training stopped by user.


In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

# ---------------------------
# 🔥 11. Visualization Function
# ---------------------------
def visualize_predictions(model, loader, num_images=5):
    """Displays random images with true and predicted labels."""
    model.eval()

    # Collect a batch of test images
    images, labels = next(iter(loader))
    images, labels = images.to(device), labels.to(device)

    # Randomly select images
    idx = random.sample(range(len(images)), num_images)

    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))

    with torch.no_grad():
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

    # Display images with true and predicted labels
    for i, ax in enumerate(axes):
        img = images[idx[i]].cpu().permute(1, 2, 0).numpy()  # Convert to (H, W, C) for display
        img = np.clip(img * 0.225 + 0.45, 0, 1)  # De-normalize for visualization

        ax.imshow(img)
        true_label = class_names[labels[idx[i]].item()]
        pred_label = class_names[preds[idx[i]].item()]

        color = "green" if true_label == pred_label else "red"
        ax.set_title(f"True: {true_label}\nPred: {pred_label}", color=color)
        ax.axis("off")

    plt.tight_layout()
    plt.show()

# ---------------------------
# 🔥 12. Call the Visualization Function
# ---------------------------
print("\n📊 Displaying 5 Random Test Images with Predictions...")
visualize_predictions(model, test_loader)
